In [1]:
from web3 import Web3
import json
web3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
assert web3.isConnected()
SELECT_COMMIT = 0
COMMENT_MESSAGE = "good"

In [2]:
with open ("contract/abi.json") as f:
    contract_abi = f.read()
with open("config.json") as f:
    config = json.load(f)
contract_address = config["contract_address"]
contract = web3.eth.contract(address=contract_address, abi=contract_abi)

In [3]:
private_key = "0x22aabb811efca4e6f4748bd18a46b502fa85549df9fa07da649c0a148d7d5530"
account = web3.eth.account.from_key(private_key)
web3.eth.default_account = account.address
print("address : %s" % account.address)
nonce = web3.eth.getTransactionCount(account.address)

address : 0xa508dD875f10C33C52a8abb20E16fc68E981F186


In [9]:
commits = contract.functions.get_all_commit_ipfs().call()
for idx , c in enumerate(commits):
    print ("%d. %s" % (idx+1 , c))

1. QmUL7wDowvNk3y7KeEYFAATmz43727FwXKhBJJrqQu813a


In [11]:
tx_params = {
    'nonce': nonce,
    'gasPrice': web3.eth.gas_price,
    'gas': 5000000, 
}

In [12]:
new_comment_tx = contract.functions.new_comment(SELECT_COMMIT,COMMENT_MESSAGE).buildTransaction(tx_params)
signed_tx = web3.eth.account.sign_transaction(new_comment_tx, private_key)
try:
    tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print ("tx_receipt : %s" % tx_receipt)
except Exception as e:
    print (str(e).split("\'")[3])

tx_receipt : AttributeDict({'transactionHash': HexBytes('0xbb2394233ad4b1195b5b63a273b849ba12f28e913d531cb7c32b0dc160df7f55'), 'transactionIndex': 0, 'blockHash': HexBytes('0x70ab3d2274202778e40f2a1e34f23d194246767df4008cacdf259493903ff067'), 'blockNumber': 154, 'from': '0xa508dD875f10C33C52a8abb20E16fc68E981F186', 'to': '0x1bf51d06070d22b2B66F06DAeeB96A8708D5090A', 'gasUsed': 105933, 'cumulativeGasUsed': 105933, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [16]:
SELECT_PROJECT = 0
comments = contract.functions.get_comments(SELECT_PROJECT,SELECT_COMMIT).call()
for idx,c in enumerate(comments):
    print("%d. %s" % (idx+1,c))

1. ('yihuai', 'good')
